In [209]:
import pandas as pd

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

not_fit = True



In [210]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

[nltk_data] Downloading package stopwords to /Users/AtSu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [211]:
import numpy as np
from bs4 import BeautifulSoup
from collections import defaultdict
import src.proc_data as procd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
"""
train_channel = np.zeros((df.values.shape[0], 4), dtype=object)
test_channel = np.zeros((df_test.values.shape[0], 4), dtype=object)
columns = ['entertainment', 'world', 'social-media', 'tech']


for i in range(df['Page content'].values.shape[0]):
    channel = procd.parse_channel(df['Page content'][i])
    if channel in columns:
        train_channel[i, columns.index(channel)] = 1

pd.DataFrame(data=train_channel, columns=columns).to_csv('train_channel.csv', index=False)
        
for i in range(df_test['Page content'].values.shape[0]):
    channel = procd.parse_channel(df_test['Page content'][i])
    if channel in columns:
        test_channel[i, columns.index(channel)] = 1
        
pd.DataFrame(data=test_channel, columns=columns).to_csv('test_channel.csv', index=False)
"""
"""
train_parse = np.zeros((df.values.shape[0], 5),dtype=object)

for i in range(df['Page content'].values.shape[0]):
#for i in range(5):

    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]\
            = procd.parse_html(df['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_b_parsed.csv')

train_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)

#for i in range(5):

for i in range(df_test['Page content'].values.shape[0]):
    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3] \
            = procd.parse_html(df_test['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_b_parsed.csv')
"""

"\ntrain_parse = np.zeros((df.values.shape[0], 5),dtype=object)\n\nfor i in range(df['Page content'].values.shape[0]):\n#for i in range(5):\n\n    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]            = procd.parse_html(df['Page content'][i])\n    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))\n\npd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_b_parsed.csv')\n\ntrain_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)\n\n#for i in range(5):\n\nfor i in range(df_test['Page content'].values.shape[0]):\n    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]             = procd.parse_html(df_test['Page content'][i])\n    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))\n\npd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_b_parsed.csv')\n"

In [212]:
df = pd.read_csv('train_parsed.csv')
df_test = pd.read_csv('test_parsed.csv')


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    
n_topics = 5
n_top_words = 15
n_features = 2**10
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                tokenizer=tokenizer_stem_nostop)
tf = tf_vectorizer.fit_transform(df['title'])

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
topic_tf = lda.fit_transform(tf)
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topic #0:
year photo twitter star use way could new instagram smartphon ipad war trailer old next
Topic #1:
world game like take show new look first tv thing help cup reveal movi song
Topic #2:
new time launch come samsung life u onlin video phone amazon internet man york big
Topic #3:
facebook appl app video iphon ad watch million one say make want top twitter report
Topic #4:
googl get know need social media news job day mobil may peopl digit glass microsoft



In [ ]:
n_twords = np.zeros((df['title'].shape[0], 1), dtype=int)
n_twords_test = np.zeros((df_test['title'].shape[0], 1), dtype=int)
for i in range(n_twords.shape[0]):
    n_twords[i] = len(tokenizer_stem_nostop(df['title'][i]))
for i in range(n_twords_test.shape[0]):
    n_twords_test[i] = len(tokenizer_stem_nostop(df_test['title'][i]))

In [ ]:
from scipy.sparse import hstack
from sklearn.preprocessing import PolynomialFeatures

train_weekday = pd.read_csv('train_weekday.csv').values
train_time = pd.read_csv('train_time.csv').values
train_channel = pd.read_csv('train_channel.csv').values

poly_time = PolynomialFeatures(degree=2)

X_train_aug = np.concatenate((train_channel, train_weekday, train_time), axis=1)
X_train_aug = poly_time.fit_transform(X_train_aug)[:, 1:]
X_train_aug = np.concatenate((X_train_aug, topic_tf), axis=1)


In [ ]:
from sklearn.model_selection import train_test_split
# hold out testing set
y_train = df['Popularity']
X_htrain, X_htest, y_htrain, y_htest = train_test_split(X_train_aug, y_train, test_size=0.3, random_state=0)


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

lr = LogisticRegression(C=1.0, random_state=0)


gnb  = GaussianNB() 

lr.fit(X_htrain, y_htrain)

pred = lr.predict(X_htrain)
pred_t = lr.predict(X_htest)
score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

# best case here
print("LR score (train): %f, (test): %f"%(score_train, score_test))
candidates = []
candidates.append({'clf':lr, 'C':3.0, 'score':0.557699})

gnb.fit(X_train_aug, y_train)
pred = gnb.predict(X_train_aug)
pred_t = gnb.predict(X_htest)
score_train = metrics.accuracy_score(y_train, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

# best case here
print("GNB score (train): %f, (test): %f"%(score_train, score_test))


lr.fit(X_train_aug, y_train)

pred = lr.predict(X_train_aug)
pred_t = lr.predict(X_htest)
score_train = metrics.accuracy_score(y_train, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

# best case here
print("LR score (train): %f, (test): %f"%(score_train, score_test))
candidates = []
candidates.append({'clf':lr, 'C':1.0, 'score':0.557699})
"""
bag = BaggingClassifier(base_estimator=lr, n_estimators=15, max_samples=0.7, max_features=0.8, oob_score=True)

bag.fit(X_train_aug, y_train)

pred = bag.predict(X_train_aug)
pred_t = bag.predict(X_htest)
score_train = metrics.accuracy_score(y_train, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

# best case here
print("voter score (train): %f, (test): %f"%(score_train, score_test))
candidates.append({'clf': bag, 'C':1.0, 'score':0.561196})
"""

In [ ]:
test_weekday = pd.read_csv('test_weekday.csv').values
test_time = pd.read_csv('test_time.csv').values
test_channel = pd.read_csv('test_channel.csv').values

X_test_con = np.concatenate((test_channel, test_weekday, test_time), axis=1)

X_test_con = poly_time.transform(X_test_con)[:,1:]
X_test_con = np.concatenate((X_test_con, topic_test), axis=1)


In [ ]:

for cand in candidates:
    predict = cand['clf'].predict(X_test_con)
    output = np.zeros((X_test_con.shape[0], 2), dtype=int)
    output[:, 0] = df_test['Id']
    output[:, 1] = predict
    df_output = pd.DataFrame(data=output, columns=['Id', 'Popularity'])
    df_output.to_csv('test___%f.csv' % (cand['C']), index=False)